In [ ]:
import requests
import pandas as pd

In [ ]:
!pip install fastapi[all] pyngrok

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

app = FastAPI()

# Allow CORS for frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


# Request Model
class AnalyzeRequest(BaseModel):
    task: str
    employee_data: list

#Creating Custom API endpoint
@app.post("/analyze")
def analyze_task(req: AnalyzeRequest):
    df = pd.json_normalize(req.employee_data)
    print(df)
    # project_df = pd.DataFrame(req.project_data)
    reply = build_prompt(req.task, df)
    return {"result": reply}


# Building Custom Prompt
def build_prompt(task,df):
  # Generated Payload for performance
    print("Inside build prompt")
    payload_performance = {
        "model": "gpt-4.1",
        "messages": [
          {
            "role": "developer",
            "content": "You are a helpful assistant."
          },
          {
            "role": "user",
            "content": df.to_string(index=False) + "\n"+"Based on provided data, Show analytics regarding performance for each employee and decribe performance in brief"
          }
        ]
      }

    # Generated Payload for availability
    payload_availability = {
        "model": "gpt-4.1",
        "messages": [
          {
            "role": "developer",
            "content": "You are a helpful assistant."
          },
          {
            "role": "user",
            "content": df.to_string(index=False) + "\n"+"Based on provided data, Show when employees will be free from their tasks"
          }
        ]
      }

    payload_staff_utilization = {
        "model": "gpt-4.1",
        "messages": [
          {
            "role": "developer",
            "content": "You are a helpful assistant."
          },
          {
            "role": "user",
            "content": df.to_string(index=False) + "\n"+"Based on provided data, Show when employees will be free from their tasks"
          }
        ]
      }
    payload = []
    if task == "performance":
      payload =  payload_performance
    elif task == "availability":
      payload =  payload_availability
    elif task == "staff_utilization":
      payload =  payload_performance

    api_url = "https://api.openai.com/v1/chat/completions"
    api_key =  "YOUR_API_KEY"
    headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
    }
    print("Before request")

    #Calling OpenAi API
    response = requests.post(api_url, headers=headers, json = payload)

    result = response.json()['choices'][0]['message']['content']
    print(result)
    return result







In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Creating Tunnel to expose custom api as Google colab does not support direct exposure
ngrok.kill()
ngrok.set_auth_token("YOUR_NGROK_AUTH_TOKEN")
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
#use this public url in frotend api in frontend/src/pages/ManagerDashboard.jsx
nest_asyncio.apply()

uvicorn.run(app, port=8000)